# Test of the clasification pipeline

In [1]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Current working directory changed to: /home/ihranicky/git/domainradar-clf/classifiers


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [17]:
create_test_parquet = True

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    input_files = [
        {'file': 'testdata/misp_2402.parquet', 'max_rows': 500, 'label': 'phishing'},
        {'file': 'testdata/benign_2312.parquet', 'max_rows': 2000, 'label': 'benign'},
        {'file': 'testdata/malware_bp.parquet', 'max_rows': 500, 'label': 'malware'},
        {'file': 'testdata/dga_2310.parquet', 'max_rows': 500, 'label': 'dga'},
    ]

    # Number of rows to select in total
    n_rows = 1000
    
    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file'])
    
    # Limit the number of rows if necessary for the first file
    if len(combined_df) > first_file_info['max_rows']:
        combined_df = combined_df.sample(n=first_file_info['max_rows'], random_state=1)
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file'])
        
        # Limit the number of rows if necessary
        if len(df) > file_info['max_rows']:
            df = df.sample(n=file_info['max_rows'], random_state=1)
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = None
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Randomly select n_rows rows from the combined DataFrame
    selected_rows = combined_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility
    
    # Save the selected rows to a new Parquet file
    selected_rows.to_parquet('testdata/sample.parquet')

/tmp/ipykernel_2196961/946921797.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([combined_df, df], ignore_index=True)


## Run classification

In [18]:
# Specify the parquet file with the dataset for classification
test_dataset = 'testdata/sample.parquet'

# Number of domain names to classify with each run of the pipeline (0 = classify all)
CHUNK_SIZE = 30

In [19]:
import pandas as pd
import numpy as np
from pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

# Determine the number of chunks
num_chunks = (len(input_df) + CHUNK_SIZE - 1) // CHUNK_SIZE if CHUNK_SIZE > 0 else 1

# Process the dataframe in chunks
for i in range(num_chunks):
    if CHUNK_SIZE > 0:
        start_idx = i * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        chunk_df = input_df[start_idx:end_idx]
    else:
        chunk_df = input_df

    # Perform your classification or processing on the working_df here
    print(f"===== Processing chunk {i+1}/{num_chunks} =====")

    chunk_without_label = chunk_df.drop(columns=['label']) # Label should not be known to classifiers
    chunk_results = clf.classify_domains(chunk_without_label)

    for result in chunk_results:
        print(result)
     
    print(f"===== Chunk {i+1}/{num_chunks} completed. =====")

    break

CNN model created
CNN model created
===== Processing chunk 1/34 =====
                             domain_name  dns_available  dns_nonzero  \
1322                 cocinortemtb.com.do       1.000000     0.725000   
975                beautysalon.melbourne       0.600000     0.125000   
3085                          irivkte.me       0.475000     0.000000   
1263                          dtx.gov.az       1.000000     0.700000   
1533                      emmavoyages.fr       0.600000     0.125000   
2508        www.mail.additionaltones.org       0.600000     0.200000   
3228                 xyhtfmk2i46hibpv.ru       0.475000     0.000000   
3308      f90j6uci30xv1t10lsz1p2vfob.net       0.475000     0.000000   
3123                ysjsjaiqqkjdgasy.net       0.475000     0.000000   
330                         44445344.xyz       0.600000     0.125000   
2359                       mail.inbox.lv       0.600000     0.200000   
297                  conecct-ter.web.app       0.600000     0.2250

/home/ihranicky/git/domainradar-clf/classifiers/preprocessor.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(-1, inplace=True)
/home/ihranicky/git/domainradar-clf/classifiers/preprocessor.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(-1, inplace=True)
/home/ihranicky/git/domainradar-clf/classifiers/Clf_dga_binary_nn.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data.fillna(-1, 

## Optional: Generate preliminary results for training the final aggregation classifier

In [20]:
import pandas as pd
import numpy as np
from pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

preliminary_results_df = clf.generate_preliminary_results(input_df, output_file="preliminary_results.parquet")

CNN model created
CNN model created


/home/ihranicky/git/domainradar-clf/classifiers/preprocessor.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(-1, inplace=True)
/home/ihranicky/git/domainradar-clf/classifiers/preprocessor.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(-1, inplace=True)
/home/ihranicky/git/domainradar-clf/classifiers/Clf_dga_binary_nn.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data.fillna(-1, 

In [21]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result
1322,cocinortemtb.com.do,1.000000,0.725000,1.000000,0.458333,1.000000,1.000000,0.833333,0.291667,1.000000,0.722222,benign,0.000000,0.023330,0.000000,0.000002,0.000002
975,beautysalon.melbourne,0.600000,0.125000,0.041667,0.000000,0.625000,0.125000,0.833333,0.000000,1.000000,0.000000,benign,0.000000,0.001260,0.000000,0.000195,0.000000
3085,irivkte.me,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,0.000422,0.106494,0.000235,0.698570,0.548605
1263,dtx.gov.az,1.000000,0.700000,1.000000,0.750000,1.000000,0.875000,0.833333,0.208333,1.000000,0.944444,benign,0.000000,0.000076,0.000000,0.000000,0.069551
1533,emmavoyages.fr,0.600000,0.125000,0.041667,0.000000,0.625000,0.125000,0.833333,0.000000,1.000000,0.000000,benign,0.000000,0.713631,0.000000,0.939006,0.138013
2508,www.mail.additionaltones.org,0.600000,0.200000,1.000000,0.750000,1.000000,0.625000,0.958333,0.583333,1.000000,0.666667,malware,1.000000,0.999970,1.000000,0.999996,0.000000
3228,xyhtfmk2i46hibpv.ru,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,0.000000,0.543204,0.000000,0.999739,0.999996
3308,f90j6uci30xv1t10lsz1p2vfob.net,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,0.000000,0.003445,0.000000,0.997792,0.999943
3123,ysjsjaiqqkjdgasy.net,0.475000,0.000000,0.041667,0.000000,0.625000,0.000000,0.833333,0.000000,1.000000,0.000000,dga,0.084735,0.004902,0.005993,0.990987,0.999202
330,44445344.xyz,0.600000,0.125000,0.041667,0.000000,0.625000,0.125000,1.000000,0.416667,1.000000,0.000000,phishing,0.000000,0.999778,0.000000,0.998906,0.030973


In [22]:
preliminary_results_df[["domain_name", "label", "phishing_cnn_result", "phishing_lgbm_result", "malware_cnn_result", "malware_xgboost_result", "dga_binary_nn_result"]]

,domain_name,label,phishing_cnn_result,phishing_lgbm_result,malware_cnn_result,malware_xgboost_result,dga_binary_nn_result
1322,cocinortemtb.com.do,benign,0.000000,0.023330,0.000000,0.000002,0.000002
975,beautysalon.melbourne,benign,0.000000,0.001260,0.000000,0.000195,0.000000
3085,irivkte.me,dga,0.000422,0.106494,0.000235,0.698570,0.548605
1263,dtx.gov.az,benign,0.000000,0.000076,0.000000,0.000000,0.069551
1533,emmavoyages.fr,benign,0.000000,0.713631,0.000000,0.939006,0.138013
2508,www.mail.additionaltones.org,malware,1.000000,0.999970,1.000000,0.999996,0.000000
3228,xyhtfmk2i46hibpv.ru,dga,0.000000,0.543204,0.000000,0.999739,0.999996
3308,f90j6uci30xv1t10lsz1p2vfob.net,dga,0.000000,0.003445,0.000000,0.997792,0.999943
3123,ysjsjaiqqkjdgasy.net,dga,0.084735,0.004902,0.005993,0.990987,0.999202
330,44445344.xyz,phishing,0.000000,0.999778,0.000000,0.998906,0.030973
